In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model, metrics

In [21]:
income= pd.read_csv("../data/raw/income_and_pop.csv")
crime = pd.read_csv("../data/curated/crime_data.csv")

In [22]:
crime = crime.rename({'Offence Count': 'offence_count'}, axis=1)

In [23]:
crime

,Year,SA2_codes,Postcode,offence_count
0,2013,206011106,3057,156
1,2013,206011107,3052,552
2,2013,206011107,3055,229
3,2013,206011109,3044,514
4,2013,206011495,3056,732
...,...,...,...,...
1795,2022,214011370,3201,675
1796,2022,214011371,3199,2994
1797,2022,214011372,3200,437
1798,2022,214011374,3910,344


In [24]:
income = income.drop('geometry', axis=1)
income

,SA2_CODE_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Median_tot_hhd_inc_weekly,total_population,SA2_NAME21
0,201011001,865,370,2218,1952,13320,Alfredton
1,201011002,842,313,2276,1573,11092,Ballarat
2,201011005,805,330,2270,1927,6041,Buninyong
3,201011006,775,360,1855,1627,8898,Delacombe
4,201011007,802,350,2236,2065,3457,Smythes Creek
...,...,...,...,...,...,...,...
466,217031476,681,285,1522,1167,3903,Otway
467,217041477,769,200,1905,1513,6137,Moyne - East
468,217041478,758,280,1911,1522,8883,Moyne - West
469,217041479,760,300,1892,1451,19400,Warrnambool - North


In [25]:
X = income.drop('total_population', axis=1)
y= income['total_population']

In [18]:

# splitting X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                    random_state=1)
  
# create linear regression object
reg = linear_model.LinearRegression()
  
# train the model using the training sets
reg.fit(X_train, y_train)

ValueError: could not convert string to float: 'Highett (East) - Cheltenham'